<a href="https://colab.research.google.com/github/AgarwalMayank2/Face_Detection/blob/main/extracting_CNN_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading dataset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/lfw-dataset")

100%|██████████| 112M/112M [00:01<00:00, 96.0MB/s]

Extracting files...


In [2]:
print(path)

/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4


In [3]:
import pandas as pd
df1=pd.read_csv("/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/people.csv")
print(df1.shape)
print(df1.head(10))

(5758, 2)
                 name  images
0  Abdel_Madi_Shabneh     1.0
1        Abdul_Rahman     1.0
2        Abel_Pacheco     4.0
3        Adriana_Lima     1.0
4         Afton_Smith     1.0
5        Ahmad_Jbarah     1.0
6      Akhmed_Zakayev     3.0
7     Alan_Dershowitz     1.0
8   Alanis_Morissette     1.0
9    Alberto_Gonzales     1.0


In [4]:
df2=pd.read_csv("/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/peopleDevTrain.csv")
print(df2.shape)
print(df2.head(10))

(4038, 2)
                  name  images
0              AJ_Cook       1
1        Aaron_Eckhart       1
2      Aaron_Patterson       1
3        Aaron_Peirsol       4
4           Aaron_Pena       1
5         Aaron_Sorkin       2
6     Abbas_Kiarostami       1
7  Abdel_Aziz_Al-Hakim       1
8   Abdel_Madi_Shabneh       1
9  Abdel_Nasser_Assidi       2


In [5]:
df3=pd.read_csv("/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/peopleDevTest.csv")
print(df3.shape)
print(df3.head(10))

(1711, 2)
                           name  images
0                      AJ_Lamas       1
1                   Aaron_Guiel       1
2                  Aaron_Tippin       1
3                     Abba_Eban       1
4        Abdul_Majeed_Shobokshi       1
5             Abdulaziz_Kamilov       1
6                  Abdullah_Gul      19
7              Abdullatif_Sener       2
8                  Abel_Aguilar       1
9  Abid_Hamid_Mahmud_Al-Tikriti       3


In [6]:
import os
images_address="/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled"
labels=os.listdir(images_address)
paths_of_images=[]
for root, dirs, files in os.walk(images_address):
    for file in files:
        paths_of_images.append(os.path.join(root, file))

print(paths_of_images[:5])

['/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Amber_Tamblyn/Amber_Tamblyn_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Amber_Tamblyn/Amber_Tamblyn_0002.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Itzhak_Perlman/Itzhak_Perlman_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Joanne_Woodward/Joanne_Woodward_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Nikolay_Davydenko/Nikolay_Davydenko_0001.jpg']


# Extracting features

Using pretrained CNN (ResNet)

In [7]:
#This code can be used to extract pretrained CNN (ResNet) Features.
#It takes Image Path and returns 2048-dimensional feature.
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Load pre-trained ResNet-50 model
resnet = models.resnet50(pretrained=True)
# Remove the last fully connected layer
resnet = nn.Sequential(*list(resnet.children())[:-1])
# Set the model to evaluation mode
resnet.eval()

# Define a function to extract features from an image
def extract_features(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    # Add batch dimension
    image = image.unsqueeze(0)
    # Extract features
    with torch.no_grad():
        features = model(image)
    # Remove the batch dimension
    features = features.squeeze(0)
    return features

# # Example usage
# image_path = '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Amber_Tamblyn/Amber_Tamblyn_0001.jpg'  # Replace 'example_image.jpg' with your image path
# features = extract_features(image_path, resnet)
# features = features.flatten()
# print(features.shape)  # Output: torch.Size([2048])
# print(features)

# # Save the extracted features
# torch.save(features, 'extracted_features.pt')  # Save the features to a file

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 133MB/s]


In [14]:
# saving featrues
import re
import numpy as np
data_list=[]
i=0
for path in paths_of_images:
    i+=1
    matchpattern = re.search(r'/([^/]+)_\d+\.jpg$', path)
    # if matchpattern:
    #     print(matchpattern.group(1))
    features = extract_features(path, resnet)
    features = features.flatten().numpy()
    datapoint = np.append(features, matchpattern.group(1))
    data_list.append(datapoint)
    if i%10==0:
        print(i,"/",len(paths_of_images),"converted")
df = pd.DataFrame(data_list)
df

10 / 13233 converted
20 / 13233 converted
30 / 13233 converted
40 / 13233 converted
50 / 13233 converted
60 / 13233 converted
70 / 13233 converted
80 / 13233 converted
90 / 13233 converted
100 / 13233 converted
110 / 13233 converted
120 / 13233 converted
130 / 13233 converted
140 / 13233 converted
150 / 13233 converted
160 / 13233 converted
170 / 13233 converted
180 / 13233 converted
190 / 13233 converted
200 / 13233 converted
210 / 13233 converted
220 / 13233 converted
230 / 13233 converted
240 / 13233 converted
250 / 13233 converted
260 / 13233 converted
270 / 13233 converted
280 / 13233 converted
290 / 13233 converted
300 / 13233 converted
310 / 13233 converted
320 / 13233 converted
330 / 13233 converted
340 / 13233 converted
350 / 13233 converted
360 / 13233 converted
370 / 13233 converted
380 / 13233 converted
390 / 13233 converted
400 / 13233 converted
410 / 13233 converted
420 / 13233 converted
430 / 13233 converted
440 / 13233 converted
450 / 13233 converted
460 / 13233 convert

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,0.057998676,1.2224677,1.5625151,0.020636946,1.1303385,0.26403067,0.958737,0.15306318,0.14015712,0.027611192,...,0.1598279,0.7167573,0.34870213,0.21117096,0.110031374,1.164732,0.06096764,0.044940215,0.057304602,Amber_Tamblyn
1,0.12865059,1.4803549,1.1338217,0.058488287,0.7934026,0.18036538,0.5873665,0.14286135,0.05171687,0.16649517,...,0.049199708,0.5239278,0.42302743,0.19781047,0.109916866,1.2816743,0.068544395,0.2025271,0.09417255,Amber_Tamblyn
2,0.016800394,0.7592781,0.19052136,0.076172404,0.25307885,0.07118152,1.1213474,0.13130029,0.031822454,0.15538448,...,0.18941303,0.21362281,0.6295689,0.59717786,0.13103516,1.6028087,0.09843259,0.0011433533,0.044447325,Itzhak_Perlman
3,0.09348315,0.7588476,0.69588095,0.41917467,0.98472714,0.2345647,1.5793715,0.12493482,0.24567129,0.7486814,...,0.2457638,0.42637277,0.18644206,0.9390815,0.14142439,1.0931586,0.11566991,0.12081951,0.09125385,Joanne_Woodward
4,0.2669828,1.0323931,1.2676535,0.058376867,0.15987292,0.15270619,2.0070534,0.43195564,0.046918012,0.98113716,...,0.09691315,0.32920605,0.3327576,0.99907863,0.17778301,1.2291324,0.10043779,0.2740116,0.25851768,Nikolay_Davydenko
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13228,0.14459106,1.0433807,0.5148761,0.15835662,0.3647982,0.13891622,1.5711364,0.3155219,0.036186192,0.3264542,...,0.1283634,0.0849461,0.575959,0.36657375,0.18569525,0.984694,0.04392411,0.099378645,0.20061031,Rita_Moreno
13229,0.12611853,0.61680526,0.5787518,0.27304265,0.37562692,0.21008608,1.3145407,0.06517392,0.021204583,0.29346487,...,0.45678586,0.5105597,0.5033467,0.5327599,0.32438165,1.6170324,0.047469705,0.19529527,0.13831241,Rita_Moreno
13230,0.24893038,0.7845206,1.0569255,0.17019346,0.022841439,0.27293044,1.0096925,0.37564492,0.05074469,0.20160986,...,0.13171114,0.9635548,0.27506483,0.38177335,0.019956127,0.7530441,0.034722738,0.0106482,0.15802397,Robin_Tunney
13231,0.08292971,1.5288603,0.18594824,0.013531867,1.4352092,0.05730391,1.6184211,0.28015924,0.53917754,0.08933866,...,0.17365749,0.45046273,0.21155743,0.9113571,0.010726766,1.0653807,0.013574603,0.025847143,0.0284882,Michael_Richards


In [15]:
df.to_csv("CNN_features_dataset.csv")